In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from feature_engineering import discretization as dc

# plt.style.use('seaborn-colorblind')
# %matplotlib inline
#from feature_cleaning import rare_values as ra

## Load Dataset

In [2]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]

data = pd.read_csv('./data/titanic.csv', usecols=use_cols)


In [3]:
data.head(3)

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,male,22.0,1,7.2500
1,1,1,female,38.0,1,71.2833
2,1,3,female,26.0,0,7.9250


In [4]:
# Note that we include target variable in the X_train 
# because we need it to supervise our discretization
# this is not the standard way of using train-test-split
X_train, X_test, y_train, y_test = train_test_split(data, data.Survived, test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((623, 6), (268, 6))

## Equal width binning
divides the scope of possible values into N bins of the same width

###  Discretización de la columna `Fare` en el dataset de Titanic

Se aplica una transformación de **binning (discretización)** a la columna `Fare` del dataset de Titanic, dividiéndola en **3 intervalos de igual tamaño (amplitud)** y codificándola con **valores numéricos ordinales**.



### Detalle paso a paso

- **`KBinsDiscretizer`**: es una clase de `scikit-learn` que transforma variables numéricas continuas en variables categóricas discretizadas.
- **`n_bins=3`**: divide los datos de `Fare` en **3 bins (intervalos)**.
- **`strategy='uniform'`**: usa bins de **igual ancho** (mismo rango entre el valor mínimo y máximo).
- **`encode='ordinal'`**: cada bin se representa con un **número entero ordinal** (`0`, `1` o `2`).



### ¿Qué efecto tiene?

Supón que los valores de `Fare` van de **0 a 100**. Esta instrucción dividirá el rango en:

| Rango de `Fare` | Valor transformado |
|------------------|--------------------|
| 0 – 33.33        | 0                  |
| 33.34 – 66.66    | 1                  |
| 66.67 – 100      | 2                  |

Cada valor de `Fare` será reemplazado por `0`, `1` o `2` según en qué bin caiga.




In [5]:
from sklearn.preprocessing import KBinsDiscretizer
enc_equal_width = KBinsDiscretizer(n_bins=3,encode='ordinal',strategy='uniform').fit(X_train[['Fare']])

In [6]:
# equal width for every bins
enc_equal_width.bin_edges_

array([array([  0.    , 170.7764, 341.5528, 512.3292])], dtype=object)

In [7]:
result = enc_equal_width.transform(X_train[['Fare']])
pd.DataFrame(result)[0].value_counts()

0
0.0    610
1.0     11
2.0      2
Name: count, dtype: int64

In [8]:
# add the new discretized variable
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_equal_width'] = enc_equal_width.transform(X_train[['Fare']])
print(X_train_copy.head(10))

     Survived  Pclass     Sex   Age  SibSp      Fare  Fare_equal_width
857         1       1    male  51.0      0   26.5500               0.0
52          1       1  female  49.0      1   76.7292               0.0
386         0       3    male   1.0      5   46.9000               0.0
124         0       1    male  54.0      0   77.2875               0.0
578         0       3  female   NaN      1   14.4583               0.0
549         1       2    male   8.0      1   36.7500               0.0
118         0       1    male  24.0      0  247.5208               1.0
12          0       3    male  20.0      0    8.0500               0.0
157         0       3    male  30.0      0    8.0500               0.0
127         1       3    male  24.0      0    7.1417               0.0


## Equal frequency binning
divides the scope of possible values of the variable into N bins, 
where each bin carries the same amount of observations

### Discretización de la columna `Fare` por frecuencia (`quantile`)

Esta transformación aplica **binning (discretización)** a la columna `Fare` del dataset de Titanic, dividiéndola en **3 grupos con la misma cantidad de observaciones** (bins con igual frecuencia), y asignando valores ordinales a cada grupo.


### Detalle paso a paso

- **`KBinsDiscretizer`**: clase de `scikit-learn` que discretiza variables numéricas continuas en variables categóricas.
- **`n_bins=3`**: divide los datos de `Fare` en **3 bins**.
- **`strategy='quantile'`**: crea bins de **igual frecuencia**, es decir, cada grupo tiene aproximadamente la misma cantidad de registros.
- **`encode='ordinal'`**: representa cada bin con un **valor numérico ordinal** (`0`, `1`, `2`).



### ¿Qué efecto tiene?

Supón que tienes 300 registros, esta transformación ordena los valores de `Fare` de menor a mayor y los divide en 3 grupos de 100 observaciones cada uno. Luego, asigna:

| Grupo | Valor transformado |
|--------|---------------------|
| 1er tercil (los valores más bajos) | 0 |
| 2do tercil (valores intermedios)   | 1 |
| 3er tercil (valores más altos)     | 2 |

Cada valor será reemplazado por `0`, `1` o `2` dependiendo de en qué **percentil** caiga.



>  A diferencia de `strategy='uniform'`, esta técnica **no divide el rango en partes iguales**, sino que garantiza que cada bin tenga una **cantidad similar de observaciones**. Es útil cuando los datos están sesgados o desbalanceados.


In [9]:
enc_equal_freq = KBinsDiscretizer(n_bins=3,encode='ordinal',strategy='quantile').fit(X_train[['Fare']])

In [10]:
# check the bin edges
enc_equal_freq.bin_edges_

array([array([  0.        ,   8.69303333,  26.2875    , 512.3292    ])],
      dtype=object)

In [11]:
# equal number of case for every bins
result = enc_equal_freq.transform(X_train[['Fare']])
pd.DataFrame(result)[0].value_counts()

0
2.0    209
0.0    208
1.0    206
Name: count, dtype: int64

In [12]:
# add the new discretized variable
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_equal_freq'] = enc_equal_freq.transform(X_train[['Fare']])
print(X_train_copy.head(10))

     Survived  Pclass     Sex   Age  SibSp      Fare  Fare_equal_freq
857         1       1    male  51.0      0   26.5500              2.0
52          1       1  female  49.0      1   76.7292              2.0
386         0       3    male   1.0      5   46.9000              2.0
124         0       1    male  54.0      0   77.2875              2.0
578         0       3  female   NaN      1   14.4583              1.0
549         1       2    male   8.0      1   36.7500              2.0
118         0       1    male  24.0      0  247.5208              2.0
12          0       3    male  20.0      0    8.0500              0.0
157         0       3    male  30.0      0    8.0500              0.0
127         1       3    male  24.0      0    7.1417              0.0


## K-means binning
using k-means to partition values into clusters

### Discretización de la columna `Fare` usando Clustering K-Means

Esta transformación aplica **binning (discretización)** a la columna `Fare` del dataset de Titanic utilizando **K-Means** como estrategia de agrupación. Los valores se agrupan en 3 clusters y luego se asigna a cada grupo un valor ordinal (`0`, `1`, `2`).



### Detalle paso a paso

- **`KBinsDiscretizer`**: clase de `scikit-learn` para transformar variables numéricas en discretas.
- **`n_bins=3`**: se forman 3 bins o grupos.
- **`strategy='kmeans'`**: usa **algoritmo de clustering K-Means** para crear los bins.
- **`encode='ordinal'`**: representa cada bin con un número entero ordinal (`0`, `1`, `2`).



### ¿Cómo funciona esta estrategia?

- En lugar de dividir por rango (`uniform`) o por cantidad de observaciones (`quantile`), agrupa los valores de `Fare` en **clusters que minimizan la varianza interna del grupo**.
- Los centros de los clusters se calculan automáticamente.
- Es útil cuando los valores de `Fare` tienen **distribuciones no uniformes o multimodales**.

### ¿Qué efecto tiene?

Por ejemplo, si los valores de `Fare` están agrupados naturalmente en tres niveles de precios (económico, medio, caro), K-Means puede descubrir esos niveles automáticamente.

| Cluster de Fare | Valor transformado |
|------------------|--------------------|
| Agrupación 1 (tarifas económicas) | 0 |
| Agrupación 2 (tarifas medias)     | 1 |
| Agrupación 3 (tarifas altas)      | 2 |

Cada valor de `Fare` será reemplazado según el cluster al que pertenezca.


> `strategy='kmeans'` puede ser más robusta para encontrar **agrupaciones naturales** en datos reales, especialmente si hay **outliers o sesgos** en la variable.


In [13]:
enc_kmeans = KBinsDiscretizer(n_bins=3,encode='ordinal',strategy='kmeans').fit(X_train[['Fare']])

In [14]:
# check the bin edges
enc_kmeans.bin_edges_

array([array([  0.        ,  93.5271531 , 338.08506324, 512.3292    ])],
      dtype=object)

In [15]:
result = enc_kmeans.transform(X_train[['Fare']])
pd.DataFrame(result)[0].value_counts()

0
0.0    587
1.0     34
2.0      2
Name: count, dtype: int64

In [16]:
# add the new discretized variable
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_kmeans'] = enc_kmeans.transform(X_train[['Fare']])
print(X_train_copy.head(10))

     Survived  Pclass     Sex   Age  SibSp      Fare  Fare_kmeans
857         1       1    male  51.0      0   26.5500          0.0
52          1       1  female  49.0      1   76.7292          0.0
386         0       3    male   1.0      5   46.9000          0.0
124         0       1    male  54.0      0   77.2875          0.0
578         0       3  female   NaN      1   14.4583          0.0
549         1       2    male   8.0      1   36.7500          0.0
118         0       1    male  24.0      0  247.5208          1.0
12          0       3    male  20.0      0    8.0500          0.0
157         0       3    male  30.0      0    8.0500          0.0
127         1       3    male  24.0      0    7.1417          0.0


## Discretisation with Decision Tree
using a decision tree to identify the optimal splitting points that would determine the bins

### Discretización de la variable `Fare` usando un Árbol de Decisión Supervisado

Esta instrucción aplica una **discretización supervisada** sobre la variable `Fare` utilizando un árbol de decisión como mecanismo para encontrar los puntos de corte óptimos según la variable objetivo (`y_train`).

El árbol de decisión en DiscretizeByDecisionTree no predice como tal, sino que se usa como herramienta para encontrar puntos óptimos de corte en una sola variable (en este caso Fare) que ayuden a separar bien la variable objetivo (y_train, por ejemplo, si alguien sobrevivió o no en Titanic).

¿Por qué usar un árbol para discretizar?
Porque:

Busca puntos de corte informativos, no arbitrarios.

Reduce la complejidad sin perder capacidad predictiva.

Captura relaciones no lineales entre una feature y el target.

In [17]:
enc1 = dc.DiscretizeByDecisionTree(col='Fare',max_depth=2).fit(X=X_train,y=y_train)

In [18]:
enc1.tree_model

DecisionTreeClassifier(max_depth=2)

In [19]:
data1 = enc1.transform(data)

In [20]:
# see how the new column Fare_tree_discret is distributed
# the values are corresponding to the proba of the prediction by the tree
print(data1.head(5))

# the unique value of the discretisized column
print(data1.Fare_tree_discret.unique())

   Survived  Pclass     Sex   Age  SibSp     Fare  Fare_tree_discret
0         0       3    male  22.0      1   7.2500           0.107143
1         1       1  female  38.0      1  71.2833           0.442308
2         1       3  female  26.0      0   7.9250           0.255319
3         1       1  female  35.0      1  53.1000           0.442308
4         0       3    male  35.0      0   8.0500           0.255319
[0.10714286 0.44230769 0.25531915 0.74626866]


In [21]:
# see how the bins are cut
# because we use a tree with max-depth of 2, we have at most 2*2=4 bins generated by the tree
col='Fare'
bins = pd.concat([data1.groupby([col+'_tree_discret'])[col].min(),
                  data1.groupby([col+'_tree_discret'])[col].max()], axis=1)
print(bins)

# all values between 0 to 7.5208 in the original variable 'Fare' 
# are given new value 0.107143 in the new column 'Fare_tree_discret'
# and so on

                      Fare      Fare
Fare_tree_discret                   
0.107143            0.0000    7.5208
0.255319            7.5500   10.5167
0.442308           11.1333   73.5000
0.746269           75.2500  512.3292


## Discretisation with ChiMerge
supervised hierarchical bottom-up (merge) method that locally exploits the chi-square criterion to decide whether two adjacent intervals are similar enough to be merged

## ¿Qué es `ChiMerge`?

**ChiMerge** es una técnica para **agrupar valores numéricos en rangos (bins)** usando una prueba estadística llamada **Chi-cuadrado**, que mide si hay relación entre una variable numérica y una variable objetivo (como “¿sobrevivió?” en el Titanic).



## ¿Para qué sirve?

Transforma una variable como `Fare` (precio del boleto) en **categorías** que ayudan a predecir otra columna, por ejemplo:

- **Boletos baratos**
- **Boletos medios**
- **Boletos caros**

## ¿Cómo funciona?

Imaginemos que somos los administradores. Tenemos muchos precios de boletos:

| Fare  | Survived |
|-------|----------|
| 7.25  | 0        |
| 8.05  | 0        |
| 71.28 | 1        |
| 512.3 | 1        |

El algoritmo se pregunta:

> "¿Estos precios de boleto tienen un comportamiento similar respecto a si alguien sobrevivió?"

### Por ejemplo, si la respuesta es **sí**, los agrupa.
- 7.25 y 8.05 (ambos no sobrevivieron) → **mismo grupo**

### Si la respuesta es **no**, los mantiene separados.
- 71.28 y 512.3 (ambos sobrevivieron) → **otro grupo**


In [22]:
enc3 = dc.ChiMerge(col='Fare',num_of_bins=5).fit(X=X_train,y='Survived')

Interval for variable Fare
  variable       interval  flag_0  flag_1
0     Fare     -inf,7.875    94.0    28.0
1     Fare   7.875,7.8792     0.0     3.0
2     Fare  7.8792,7.8958    25.0     1.0
3     Fare    7.8958,73.5   245.0   160.0
4     Fare          73.5+    17.0    50.0


In [23]:
# the bins boundary created by ChiMerge

enc3.bins

[-0.1, 7.875, 7.8792, 7.8958, 73.5, 512.3292]

In [24]:
data3 = enc3.transform(data)

In [25]:
print(data3.head(5))

   Survived  Pclass     Sex   Age  SibSp     Fare    Fare_chimerge
0         0       3    male  22.0      1   7.2500  (-0.101, 7.875]
1         1       1  female  38.0      1  71.2833    (7.896, 73.5]
2         1       3  female  26.0      0   7.9250    (7.896, 73.5]
3         1       1  female  35.0      1  53.1000    (7.896, 73.5]
4         0       3    male  35.0      0   8.0500    (7.896, 73.5]


In [26]:
# all values are grouped into 5 intervals
data3.Fare_chimerge.unique()

[(-0.101, 7.875], (7.896, 73.5], (73.5, 512.329], (7.875, 7.879], (7.879, 7.896]]
Categories (5, interval[float64, right]): [(-0.101, 7.875] < (7.875, 7.879] < (7.879, 7.896] < (7.896, 73.5] < (73.5, 512.329]]